In [ ]:
!pip install transformers
!pip install torch

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

In [ ]:
import csv
import json

csv_file_path = 'medquad.csv'
json_file_path = 'output.json'

# Read CSV file and convert to JSON
data = []
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        data.append(row)

# Write JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [ ]:
with open(r"output.json", "r") as read_file:
    mydata = json.load(read_file)

In [ ]:
model_name = 'deepset/roberta-base-squad2'

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
for i, item in enumerate(mydata[:100]):
    # Tokenize the question and context
    inputs = tokenizer(
        item['question'],
        item['answer'],
        max_length=512,  # Set the maximum sequence length
        truncation=True,
        return_tensors='pt'
    )

    # Check if the tokenized input exceeds the maximum length
    if inputs['input_ids'].size(1) > 512:
        print(f"Skipping example {i + 1} due to excessive length")
        continue

    # Model prediction
    outputs = model(**inputs)

    # Extract answer indices
    answer_start_idx = torch.argmax(outputs.start_logits)
    answer_end_idx = torch.argmax(outputs.end_logits)

    # Decode the answer tokens
    answer_tokens = inputs['input_ids'][0, answer_start_idx:answer_end_idx + 1]
    answer = tokenizer.decode(answer_tokens)

    # Print the result
    print("Question: {}\nAnswer: {}".format(item['question'], answer))

Question: What is (are) Glaucoma ?
Answer:  a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness
Question: What causes Glaucoma ?
Answer:  a leading cause of blindness
Question: What are the symptoms of Glaucoma ?
Answer:  Glaucoma can develop in one or both eyes
Question: What are the treatments for Glaucoma ?
Answer:  medication and surgery
Question: What is (are) Glaucoma ?
Answer:  a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness
Question: What is (are) Glaucoma ?
Answer: <s>
Question: What is (are) Glaucoma ?
Answer: Open-angle glaucoma
Question: Who is at risk for Glaucoma? ?
Answer:  African-Americans over age 40  - everyone over age 60
Question: How to prevent Glaucoma ?
Answer:  we do not know
Question: What are the symptoms of Glaucoma ?
Answer:  no symptoms
Question: What are the treatments for Glaucoma ?
Answer:  medicines, laser surgery, conventional surgery, or a combination of a